In [132]:
import pandas as pd
import numpy as np
import localization as lx

DF = pd.read_csv('Trilateration-Tests/wifi-232,371,125,test9.csv') 
groundtruth = (-232,371,125)

#Anchor positions
FTM_1_POS = (-251,-172,66)
FTM_2_POS = (377,146,70)
FTM_3_POS = (-226,738,89)
FTM_4_POS = (256,628,89)

#RSSI LOC WiFi
d0 = 1 #d0 distance
#ref_rssi = -32.08 #rssi at d0
#ref_rssi = -27
#var_rssi = 0.45 #var wifi rssi
ref_rssi = -41.25
var_rssi = 0.44 #var wifi rssi
rssi_distance = 0
path_loss = 3
params = (d0, ref_rssi, path_loss, var_rssi)

#tof_calib_1meter = 5336 #cm
tof_calib_1meter = 5473 #cm

#RSSI distance estimation
def estimate_rssi_distance(power_received):

    d_ref = params[0] # reference distance
    power_ref = params[1] # mean received power at reference distance
    path_loss_exp = params[2] # path loss exponent
    stdev_power = np.sqrt(params[3]) # standard deviation of received power

    uncertainty = 2*stdev_power # uncertainty in RSS corresponding to 95.45% confidence

    d_est = d_ref*(10**(-(power_received - power_ref)/(10*path_loss_exp)))
    d_min = d_ref*(10**(-(power_received - power_ref + uncertainty)/(10*path_loss_exp)))
    d_max = d_ref*(10**(-(power_received - power_ref - uncertainty)/(10*path_loss_exp)))

    return (np.round(d_est,2), np.round(d_min,2), np.round(d_max,2))

point = 0
DF['POINT'] = 0
for i in range(len(DF)):
    if i > 0: 
        if DF.iloc[i]['timestamp'] == DF.iloc[i-1]['timestamp']:
            #DF[i]['POINT'] = point
            DF.iloc[i, DF.columns.get_loc('POINT')] = point
        else:
            point += 1
            DF.iloc[i, DF.columns.get_loc('POINT')] = point
    elif i == 0:
        DF.iloc[i, DF.columns.get_loc('POINT')] = 0

In [133]:
point = 0
max_point = DF.iloc[-1]['POINT']
for i in range(DF.iloc[-1]['POINT']+1):
    PT = DF['POINT']==point
    DF_PT = DF[PT]
    print(DF_PT[['timestamp','SSID','RSSI','POINT','DIST']])
    #Trilateration
    P_ToF=lx.Project(mode='3D',solver='LSE')
    P_RSSI=lx.Project(mode='3D',solver='LSE')
    #Anchors
    P_ToF.add_anchor('FTM_1',FTM_1_POS)
    P_ToF.add_anchor('FTM_2',FTM_2_POS)
    P_ToF.add_anchor('FTM_3',FTM_3_POS)
    P_ToF.add_anchor('FTM_4',FTM_4_POS)
    P_RSSI.add_anchor('FTM_1',FTM_1_POS)
    P_RSSI.add_anchor('FTM_2',FTM_2_POS)
    P_RSSI.add_anchor('FTM_3',FTM_3_POS)
    P_RSSI.add_anchor('FTM_4',FTM_4_POS)
    #Gateway
    tof,label=P_ToF.add_target()
    rssi,label=P_RSSI.add_target()

    if len(DF_PT)>2:
        #print("localizatioin in progress for point "+str(point))
        for i in range(len(DF_PT)):
            tof.add_measure(DF_PT.iloc[i][1],(DF_PT.iloc[i][6]-tof_calib_1meter))
            rssi.add_measure(DF_PT.iloc[i][1],(estimate_rssi_distance(DF_PT.iloc[i][3])[0]*100))

        P_ToF.solve()
        P_RSSI.solve()

        #print("ToF localization: "+str(tof.loc)+", error is: "+str(abs(tof.loc[0]-groundtruth[0])) + "," +str(abs(tof.loc[1]-groundtruth[1])) + "," +str(abs(tof.loc[2]-groundtruth[2])))

        #print("RSSI localization: "+str(rssi.loc)+", error is: "+str(abs(rssi.loc[0]-groundtruth[0])) + "," +str(abs(rssi.loc[1]-groundtruth[1])) + "," +str(abs(rssi.loc[2]-groundtruth[2])))
        print("Groundtruth: "+str(groundtruth))
        print("ToF localization: "+str(tof.loc))
        print("RSSI localization: "+str(rssi.loc))

    point += 1


                    timestamp   SSID  RSSI  POINT  DIST
0  2023-03-21 17:05:44.963520  FTM_2 -42.0      0  6010
1  2023-03-21 17:05:44.963520  FTM_3 -46.0      0  5580
2  2023-03-21 17:05:44.963520  FTM_4 -54.0      0  5940
LSE Geolocating...
LSE Geolocating...
Groundtruth: (-232, 371, 125)
ToF localization: p(-143.90243620470739,552.8109695063869,82.04024604529592)
RSSI localization: p(87.46375306445745,426.8815809606688,78.98159546212193)
                    timestamp   SSID  RSSI  POINT  DIST
3  2023-03-21 17:05:46.500492  FTM_3 -48.0      1  5760
4  2023-03-21 17:05:46.500492  FTM_1 -45.0      1  5730
5  2023-03-21 17:05:46.500492  FTM_2 -45.0      1  5960
6  2023-03-21 17:05:46.500492  FTM_4 -53.0      1  5910
LSE Geolocating...
LSE Geolocating...
Groundtruth: (-232, 371, 125)
ToF localization: p(-128.12705477774367,287.2581507968038,77.72625325818267)
RSSI localization: p(7.213440560266537,289.2025831622277,77.04432530761018)
                     timestamp   SSID  RSSI  POINT  DI

In [ ]:
import numpy as np

# Kalman filter initialization
x = np.array([[1.0], [0.0]])  # initial state (distance and velocity)
P = np.diag((1.0, 1.0))      # initial uncertainty
Q = np.diag((0.01, 0.01))    # process noise
R = np.array([[10.0]])       # measurement noise

# WiFi RSSI to distance conversion
def rssi_to_distance(rssi):
    A = -47.0  # signal strength at 1 meter
    n = 2.7    # signal attenuation exponent
    return 10**((A-rssi)/(10*n))

# Kalman filter update
def kalman_filter(z):
    # Predict
    x = np.dot(F, x)
    P = np.dot(np.dot(F, P), F.T) + Q

    # Update
    y = z - np.dot(H, x)
    S = np.dot(np.dot(H, P), H.T) + R
    K = np.dot(np.dot(P, H.T), np.linalg.inv(S))
    x = x + np.dot(K, y)
    P = np.dot((np.eye(2) - np.dot(K, H)), P)

    return x, P

# Main loop
for rssi in rssi_data:
    distance = rssi_to_distance(rssi)
    F = np.array([[1.0, 1.0], [0.0, 1.0]])  # state transition matrix
    H = np.array([[distance, 0.0]])         # measurement matrix
    x, P = kalman_filter(distance)
    print("Estimated distance: ", x[0][0])

In [109]:
import requests
from bs4 import BeautifulSoup
#import telegram

# Set up the Telegram bot
#bot_token = 'YOUR_BOT_TOKEN_HERE'
#chat_id = 'YOUR_CHAT_ID_HERE'
#bot = telegram.Bot(token=bot_token)

# Scrape the SMHI website for the temperature in Luleå
url = 'https://www.smhi.se/vader/observationer/observationer#ws=wpt-a,proxy=wpt-a,tab=vader,param=t,stationid=162860,type=weather'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
lulea_temp = soup.find('a', {'title': 'Luleå-Kallax Flygplats'}).parent.parent.find_all('td')[-1].text

# Publish the temperature to the Telegram bot
#bot.send_message(chat_id=chat_id, text=f'Temperature in Luleå: {lulea_temp}°C')


AttributeError: 'NoneType' object has no attribute 'parent'